In [56]:
import torch
device = "cuda"

In [57]:
text = open("input.txt", "r").read()

In [58]:
print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [59]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [60]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s ]
decode = lambda l : "".join([itos[i] for i in l])

print(encode("hi there"))
print(decode([46, 47, 1, 58, 46, 43, 56, 43]))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [61]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [62]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [63]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [64]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    print(f"input: {x[:t+1]} , output: {y[t]}")

input: tensor([18]) , output: 47
input: tensor([18, 47]) , output: 56
input: tensor([18, 47, 56]) , output: 57
input: tensor([18, 47, 56, 57]) , output: 58
input: tensor([18, 47, 56, 57, 58]) , output: 1
input: tensor([18, 47, 56, 57, 58,  1]) , output: 15
input: tensor([18, 47, 56, 57, 58,  1, 15]) , output: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) , output: 58


In [65]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device) , y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] th

In [66]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModle(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets = None):
        logits = self.token_embedding_table(idx) # (B,T,C)
        if(targets is None):
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(-1,C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits,targets)
        return logits , loss
    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(idx)
            logits  = logits[:,-1,:]
            probs = F.softmax(logits)
            idx_next = torch.multinomial(probs,1)
            idx = torch.cat((idx,idx_next) , dim=1)
        return idx

model = BigramLanguageModle(vocab_size)
m = model.to(device)
out , loss = m(xb,yb)
print(loss)

tensor(4.8786, device='cuda:0', grad_fn=<NllLossBackward0>)


In [67]:
idx = torch.zeros((1,1) , dtype=torch.long , device=device)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))


pYCXxfRkRZd
wc'wfNfT;OLlTEeC K
jxqPToTb?bXAUG:C-SGJO-33SM:C?YI3a
hs:LVXJFhXeNuwqhObxZ.tSVrddXlaSZaNe


C:\Users\GHASSEN\AppData\Local\Temp\ipykernel_15672\816921892.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits)


In [68]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [70]:
batch_size = 32
for steps in range(10000):
    xb , yb = get_batch("train")
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
    

2.4723217487335205


In [72]:
idx = torch.zeros((1,1) , dtype=torch.long , device=device)
print(decode(m.generate(idx,max_new_tokens=500)[0].tolist()))

C:\Users\GHASSEN\AppData\Local\Temp\ipykernel_15672\816921892.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits)



Wawice my.

HNEdarom oroup
Yowhthetof isth ble mil ndill, ath iree sengmin lat Heriliovets, and Win nghir.
Thanousel lind me l.
HAshe ce hiry:
Supr aisspllw y.
Herindu noroopetelaves
MP:

Pl, d mothakleo Windo whth eisbyo the m dourive we higend t so mower; te

AN ad nterupt f s ar igr t m:

Thiny aleronth,
Mad
Whed my o myr f-NLIERor,
Sb&y, wardsal thes ghesthidin cour ay aney Iry ts I fr y ce.
JMOn pand, bemary.
Yof 'Wour menm sora anghy t-e nomes twe ten.
Wand thot sulin s th llety ome.
I muc


##### Math Trick for later softmax:
the new embedding of each words is the average of its previous words.
to calculate that we used matrix multipilcation and the trick is to use triangle matrices.



In [73]:
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [80]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev,0)
x[0]

tensor([[-0.4405,  1.3401],
        [ 1.1232, -0.3638],
        [-2.0964, -1.8155],
        [-0.0260,  0.6927],
        [ 1.5187, -0.1360],
        [ 0.1352, -0.6181],
        [-0.5441,  0.7337],
        [-1.1863, -2.0359]])

In [96]:
tril = torch.tril(torch.ones(T,T))
wei = tril.masked_fill(tril==0 , float("-inf"))
wei = F.softmax(wei,dim=1)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [124]:

B,T,C = 4,8,32
x = torch.randn(B,T,C)
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x) # (B,T,H)
q = query(x) # (B,T,H)
v = value(x) # (B,T,H)
wei = q @ k.transpose(-2,-1) / (head_size**0.5)
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0 , float("-inf"))
wei = F.softmax(wei,dim=-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [126]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [7.2354e-01, 2.7646e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.2658e-01, 1.0872e-01, 6.4700e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [6.9475e-02, 4.5799e-01, 7.0649e-02, 4.0188e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.3575e-01, 6.4929e-02, 4.3506e-01, 1.7465e-01, 8.9612e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [7.0534e-04, 2.3176e-02, 6.8286e-04, 1.3578e-01, 8.9429e-04, 8.3876e-01,
         0.0000e+00, 0.0000e+00],
        [1.9324e-02, 2.1439e-02, 3.9943e-02, 9.5961e-03, 2.0736e-01, 7.7763e-02,
         6.2457e-01, 0.0000e+00],
        [6.5604e-02, 1.3750e-01, 3.2650e-01, 3.4503e-02, 1.8399e-01, 1.4976e-01,
         4.1817e-02, 6.0331e-02]], grad_fn=<SelectBackward0>)

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(C,head_size,bias=False)
        self.query = nn.Linear(C,head_size,bias=False)
        self.value = nn.Linear(C,head_size,bias=False)
        self.register_buffer("tril",torch.tril(torch.ones(block_size,block_size)))
    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,H)
        q = self.query(x) 
        v = self.value(x)
        wei = q @ k.transpose(-2,-1) / (head_size**0.5)
        wei = wei.masked_fill(self.tril==0 , float("-inf"))
        wei = F.softmax(wei,dim=-1)
        out = wei @ v
        return out
    

In [ ]:
n_emb = 32
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModle(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_emb)
        self.positional_embedding = nn.Embedding(block_size,n_emb)
        self.sa_head = Head(head_size)
        self.lm_head = nn.Linear(n_emb,vocab_size)
    def forward(self,idx,targets = None):
        B,T = idx.shape
        token_emb = self.token_embedding_table(idx)
        pos_emp = torch.arange(T,device=device)
        pos_emb = self.positional_embedding(pos_emp)
        x = token_emb + pos_emb
        x = self.sa_head(x)
        logits = self.lm_head(x)
        if(targets is None):
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(-1,C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits,targets)
        return logits , loss
    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            idx = idx[:,-block_size:]
            logits,loss = self(idx)
            logits  = logits[:,-1,:]
            probs = F.softmax(logits)
            idx_next = torch.multinomial(probs,1)
            idx = torch.cat((idx,idx_next) , dim=1)
        return idx

model = BigramLanguageModle(vocab_size)
m = model.to(device)
out , loss = m(xb,yb)
print(loss)